In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

In [2]:
vitals = pd.read_hdf("all_hourly_data.h5", 'vitals_labs_mean')
print(vitals.shape)

(2200954, 104)


In [3]:
interventions = pd.read_hdf("all_hourly_data.h5",'interventions')
print(interventions.shape)

(2200954, 14)


In [4]:
patients = pd.read_hdf("all_hourly_data.h5", 'patients')
print(patients.shape)

(34472, 28)


In [5]:
# Prepare labels for los > 7 task
patients["los_7"] = patients['los_icu'] > 7
patients.drop(columns=['los_icu'], inplace=True)

In [6]:
patients = patients[["gender","ethnicity","age","los_7","mort_icu"]].reset_index()

In [7]:
patients.los_7 = patients.los_7.apply(int)

In [8]:
print(f"class imbalance for length of stay prediction: {patients.los_7.mean()}")
print(f"class imbalance for icu mortality prediction: {patients.mort_icu.mean()}")

class imbalance for length of stay prediction: 0.053521698770016245
class imbalance for icu mortality prediction: 0.06558946391274077


WINDOW_SIZE determines how much information we want to use for the prediction tasks. For example, a window length of 24 means that we will use the sequence of the first 24 readings for vitals and interventions.

In [9]:
WINDOW_SIZE = 24

In [10]:
interventions = interventions.reset_index()

In [11]:
interventions = interventions[interventions.hours_in < WINDOW_SIZE]

In [12]:
# interventions.drop(columns=['level_0','index'], inplace=True)

In [13]:
interventions

,subject_id,hadm_id,icustay_id,hours_in,vent,vaso,adenosine,dobutamine,dopamine,epinephrine,isuprel,milrinone,norepinephrine,phenylephrine,vasopressin,colloid_bolus,crystalloid_bolus,nivdurations
0,3,145834,211552,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,145834,211552,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0
2,3,145834,211552,2,1,1,0,0,1,0,0,0,0,1,0,0,0,0
3,3,145834,211552,3,1,1,0,0,0,0,0,0,0,1,0,0,0,0
4,3,145834,211552,4,1,1,0,0,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200946,99999,113369,246512,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2200947,99999,113369,246512,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2200948,99999,113369,246512,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2200949,99999,113369,246512,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
vitals = vitals.reset_index()

In [15]:
vitals = vitals[vitals.hours_in < WINDOW_SIZE]

In [16]:
vitals.shape

(808539, 108)

In [17]:
vitals.columns = vitals.columns.to_flat_index()
vitals.columns = list(map(lambda x: x[0] + x[1] if x[1] == "" else x[0] + " " + x[1] , vitals.columns.tolist()))

In [18]:
vitals

,subject_id,hadm_id,icustay_id,hours_in,alanine aminotransferase mean,albumin mean,albumin ascites mean,albumin pleural mean,albumin urine mean,alkaline phosphate mean,...,total protein mean,total protein urine mean,troponin-i mean,troponin-t mean,venous pvo2 mean,weight mean,white blood cell count mean,white blood cell count urine mean,ph mean,ph urine mean
0,3,145834,211552,0,25.0,1.8,NaN,NaN,NaN,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,14.842857,NaN,7.40,5.0
1,3,145834,211552,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,145834,211552,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.26,NaN
3,3,145834,211552,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,145834,211552,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200946,99999,113369,246512,19,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2200947,99999,113369,246512,20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2200948,99999,113369,246512,21,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2200949,99999,113369,246512,22,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
vitals.isna().mean(axis=0)

subject_id                           0.000000
hadm_id                              0.000000
icustay_id                           0.000000
hours_in                             0.000000
alanine aminotransferase mean        0.968198
                                       ...   
weight mean                          0.954340
white blood cell count mean          0.883480
white blood cell count urine mean    0.995364
ph mean                              0.859961
ph urine mean                        0.978932
Length: 108, dtype: float64

In [20]:
patients

,subject_id,hadm_id,icustay_id,gender,ethnicity,age,los_7,mort_icu
0,3,145834,211552,M,WHITE,76.526792,0,0
1,4,185777,294638,F,WHITE,47.845047,0,0
2,6,107064,228232,F,WHITE,65.942297,0,0
3,9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,0,1
4,11,194540,229441,F,WHITE,50.148295,0,0
...,...,...,...,...,...,...,...,...
34467,99983,117390,286606,M,UNKNOWN/NOT SPECIFIED,78.576624,0,0
34468,99991,151118,226241,M,WHITE,47.729259,0,0
34469,99992,197084,242052,F,WHITE,65.772155,0,0
34470,99995,137810,229633,F,WHITE,88.698942,0,0


In [21]:
interventions_list = sorted(list(set(interventions.columns.tolist())-set(['subject_id', 'hadm_id', 'icustay_id', 'hours_in'])))

In [22]:
interventions_mapping = dict(zip(interventions_list, range(1, 1+len(interventions_list))))

In [23]:
interventions_mapping

{'adenosine': 1,
 'colloid_bolus': 2,
 'crystalloid_bolus': 3,
 'dobutamine': 4,
 'dopamine': 5,
 'epinephrine': 6,
 'isuprel': 7,
 'milrinone': 8,
 'nivdurations': 9,
 'norepinephrine': 10,
 'phenylephrine': 11,
 'vaso': 12,
 'vasopressin': 13,
 'vent': 14}

In [24]:
subjects = patients.subject_id.tolist()

In [26]:
los_7 = patients.los_7
mort = patients.mort_icu

In [27]:
with open("los.pkl","wb") as f:
    pickle.dump(los_7,f)
    
with open("mort.pkl","wb") as f:
    pickle.dump(mort,f)

### "Injecting" Demographic Data

In [28]:
bins= [0,16,35,60,80,100]
labels = ['child','young_adult','middle_age','old','very_old']
def bucket_age(age_column):
    bucketed = pd.cut(age_column, bins=bins, labels=labels, right=False).astype(str)
    return bucketed

In [29]:
patients.age = bucket_age(patients.age)

In [30]:
patients.age

0               old
1        middle_age
2               old
3        middle_age
4        middle_age
            ...    
34467           old
34468    middle_age
34469           old
34470      very_old
34471           old
Name: age, Length: 34472, dtype: object

In [31]:
patients[["gender","ethnicity", "age"]]

,gender,ethnicity,age
0,M,WHITE,old
1,F,WHITE,middle_age
2,F,WHITE,old
3,M,UNKNOWN/NOT SPECIFIED,middle_age
4,F,WHITE,middle_age
...,...,...,...
34467,M,UNKNOWN/NOT SPECIFIED,old
34468,M,WHITE,middle_age
34469,F,WHITE,old
34470,F,WHITE,very_old


In [32]:
l = patients[["gender","ethnicity", "age"]].values.tolist()
cat_values = set([item for sublist in l for item in sublist])
demo_mapping = dict(zip(cat_values, range(15,15 + len(cat_values))))
demographics = patients[["gender","ethnicity", "age"]].applymap(lambda x: demo_mapping[x])

In [33]:
patients[["gender","ethnicity", "age"]] = demographics

In [34]:
patients

,subject_id,hadm_id,icustay_id,gender,ethnicity,age,los_7,mort_icu
0,3,145834,211552,21,32,34,0,0
1,4,185777,294638,62,32,43,0,0
2,6,107064,228232,62,32,34,0,0
3,9,150750,220597,21,44,43,0,1
4,11,194540,229441,62,32,43,0,0
...,...,...,...,...,...,...,...,...
34467,99983,117390,286606,21,44,34,0,0
34468,99991,151118,226241,21,32,43,0,0
34469,99992,197084,242052,62,32,34,0,0
34470,99995,137810,229633,62,32,56,0,0


In [47]:
# to inform embedding table vocab size (we will use the same vocab across the columns as there are no overlaps)
print(f"number of distinct categories across demographic columns: {len(cat_values)}")

number of distinct categories across demographic columns: 49


In [154]:
interventions_demo = interventions.merge(patients,how="left", on="subject_id")

In [162]:
# these columns may have the value of 1 which will mess up the process of aggregating the interventions per time step
interventions_demo = interventions_demo.drop(["hours_in","los_7","mort_icu"],axis=1)

In [163]:
demo_seqs = []
for subject in subjects:
    interv_list = []
    tmp = interventions_demo[interventions_demo.subject_id == subject]
    for _, i in tmp.iterrows():
        interv_demo = interventions_demo.columns[i.eq(1).values].tolist()
        # if "hours_in" in interv_demo:
        #     interv_demo.remove("hours_in")
        intervention_at_timestamp = list(map(lambda x: interventions_mapping[x], interv_demo))
        interv_list.append( i[["gender","ethnicity","age"]].values.tolist() + intervention_at_timestamp)
    demo_seqs.append(interv_list)

In [165]:
with open("demograph_intervention_seq.pkl","wb") as f:
    pickle.dump(demo_seqs,f)

### Repeat for vitals

#### Handling missing values: impute with zero

In [97]:
vitals_zero = vitals.fillna(0)

In [90]:
l = patients[["gender","ethnicity", "age"]].values.tolist()
cat_values = set([item for sublist in l for item in sublist])

In [91]:
demo_mapping = dict(zip(cat_values, range(len(cat_values))))


In [94]:
demographics = patients[["gender","ethnicity", "age"]].applymap(lambda x: demo_mapping[x])

In [95]:
patients[["gender","ethnicity", "age"]] = demographics

In [98]:
vitals_zero_demo = vitals_zero.merge(patients,how="left", on="subject_id")

In [99]:
vitals_columns = [x for x in vitals.columns.tolist() if x not in ['subject_id', 'hadm_id', 'icustay_id', 'hours_in']]

In [100]:
demo_columns = ["gender","ethnicity","age"]

In [103]:
vitals_zero_demo[["subject_id"] + vitals_columns + demo_columns]

,subject_id,alanine aminotransferase mean,albumin mean,albumin ascites mean,albumin pleural mean,albumin urine mean,alkaline phosphate mean,anion gap mean,asparate aminotransferase mean,basophils mean,...,troponin-t mean,venous pvo2 mean,weight mean,white blood cell count mean,white blood cell count urine mean,ph mean,ph urine mean,gender,ethnicity,age
0,3,25.0,1.8,0.0,0.0,0.0,73.0,20.666667,69.0,0.0,...,0.0,0.0,0.0,14.842857,0.0,7.40,5.0,36,38,34
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,36,38,34
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,7.26,0.0,36,38,34
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,36,38,34
4,3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,36,38,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808534,99999,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,48,45,34
808535,99999,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,48,45,34
808536,99999,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,48,45,34
808537,99999,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,48,45,34


In [104]:
len(vitals_columns)

104

In [105]:
demo_vseqs[1].shape

(24, 107)

In [106]:
demo_vseqs = []
for subject in subjects:
    v_zero_demo = vitals_zero_demo[vitals_zero_demo.subject_id == subject]
    v_zero_demo = v_zero_demo[vitals_columns + demo_columns].values

    demo_vseqs.append(v_zero_demo)


In [107]:
len(demo_vseqs)

34472

In [108]:
with open("demograph_vitals_zero_seq.pkl","wb") as f:
    pickle.dump(demo_vseqs,f)